In [ ]:
!pwd

/Users/l_y_o/Work/flux-training/notebooks


In [151]:
# load /Users/l_y_o/Work/happyface/pseo/ai_girls_sample_images.json

images = json.load(open('/Users/l_y_o/Work/happyface/pseo/ai_girls_sample_images.json'))

# show the first image
images[0]

# show the last image
images[-1]

'https://scontent-dfw5-1.cdninstagram.com/v/t39.30808-6/480111205_614516708177388_5606094798829422550_n.jpg?stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE2Mzl4MjA0OC5zZHIuZjMwODA4LmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=100&_nc_oc=Q6cZ2QGgZgJmFnqt3zUxmN8Umaz3EXYZ2-APpmEY-AZEc40r5ZLauOSpn4XGoec1-HkX-v4&_nc_ohc=NH5kl6ffFoMQ7kNvgFjKN1G&_nc_gid=Bggjtait2N8u54unKWC_6w&edm=APoiHPcAAAAA&ccb=7-5&ig_cache_key=MzI1NzE1Njc5MTgzNTk3NDg2Nw%3D%3D.3-ccb7-5&oh=00_AYETAup4uWa1dQWQY6-XgshmJqPybmhCBDITY5Zy7nCIng&oe=67F0C627&_nc_sid=22de04'

In [153]:
import json
import time
import requests
import random
import os
import re
from typing import Optional, Dict, List, Any


def get_image_tags(
    image_url: str,
    comfy_server_url: str = "http://72.70.187.181:50346/",
    workflow_file: str = "/Users/l_y_o/Work/happyface/pseo/image-tagger-api.json",
    max_retries: int = 3,
    poll_interval: float = 2.0,
    timeout: int = 300
) -> Optional[Dict[str, Any]]:
    """
    Get tags for an image using ComfyUI workflow.
    
    Args:
        image_url: URL of the image to tag
        comfy_server_url: URL of the ComfyUI server
        workflow_file: Path to the workflow JSON file
        max_retries: Maximum number of retries for the job submission
        poll_interval: Time in seconds between status check polls
        timeout: Maximum time in seconds to wait for job completion
        
    Returns:
        Dictionary containing the tags and confidence scores, or None if tagging failed
    """
    # Load the workflow
    try:
        with open(workflow_file, 'r') as f:
            workflow = json.load(f)
    except Exception as e:
        print(f"Error loading workflow file: {str(e)}")
        return None
    
    # Modify the image URL in the workflow
    # Find the LoadImageFromUrlOrPath node
    for node_id, node in workflow.items():
        if node.get("class_type") == "LoadImageFromUrlOrPath":
            # Replace the image URL
            node["inputs"]["url_or_path"] = image_url
            break
    
    # Prepare the API request
    prompt_api_url = f"{comfy_server_url}/prompt"
    
    # Prepare the payload
    payload = {
        "prompt": workflow,
        "client_id": f"comfy_tagger_{int(time.time())}"
    }
    
    # Submit the job
    job_id = None
    for attempt in range(max_retries):
        try:
            print(f"Submitting tagging job attempt {attempt+1}/{max_retries}...")
            response = requests.post(prompt_api_url, json=payload)
            response.raise_for_status()
            response_data = response.json()
            job_id = response_data.get("prompt_id")
            if job_id:
                print(f"Successfully submitted job with ID: {job_id}")
                break
        except Exception as e:
            print(f"Attempt {attempt+1}/{max_retries} - Error submitting job: {str(e)}")
            if attempt < max_retries - 1:
                sleep_time = 2 ** attempt  # Exponential backoff
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
    
    if not job_id:
        print("Failed to submit job after all retries")
        return None
    
    # Poll for job completion using the job history endpoint
    start_time = time.time()
    job_api_url = f"{comfy_server_url}/history/{job_id}"
    queue_api_url = f"{comfy_server_url}/queue"
    
    print(f"Waiting for job {job_id} to complete...")
    
    while time.time() - start_time < timeout:
        try:
            # Check queue status to see if our job is running
            queue_response = requests.get(queue_api_url)
            if queue_response.status_code == 200:
                queue_data = queue_response.json()
                running = queue_data.get("running", [])
                queue_remaining = queue_data.get("queue_remaining", 0)
                
                if running:
                    for job in running:
                        # Check if our job is running
                        if job.get("prompt_id") == job_id:
                            progress = job.get("progress", 0)
                            print(f"Job running: {progress*100:.1f}% complete")
                elif queue_remaining > 0:
                    print(f"Job queued. Position in queue: {queue_remaining}")
            
            # Get job status from history
            job_response = requests.get(job_api_url)
            
            # Skip if job not found yet
            if job_response.status_code == 404:
                print(f"Job {job_id} not found in history yet. Waiting...")
                time.sleep(poll_interval)
                continue
                
            job_response.raise_for_status()
            job_data = job_response.json()

            if job_data == {}:
                print(f"Job {job_id} not found in history yet. Waiting...")
                time.sleep(poll_interval)
                continue
            
            # The response has the job_id as the key
            if job_id in job_data:
                job_info = job_data[job_id]
                
                # Check for outputs
                outputs = job_info.get("outputs", {})
                if outputs:
                    # Check output from WD14 Tagger node (node 1)
                    if "1" in outputs and outputs["1"]:
                        if "tags" in outputs["1"]:
                            tags_data = outputs["1"]["tags"]
                            print(f"Successfully extracted image tags")
                            
                            # Parse tags output
                            # The output format may vary based on the tagger implementation
                            # Here we handle common format: a string of comma-separated tags
                            if isinstance(tags_data, str):
                                tags_list = [tag.strip() for tag in tags_data.split(',')]
                                return {"tags": tags_list}
                            elif isinstance(tags_data, list):
                                return {"tags": tags_data}
                            elif isinstance(tags_data, dict):
                                return tags_data
                            else:
                                print(f"Unexpected tags format: {type(tags_data)}")
                                return {"raw_tags": tags_data}
                
                # Check if job is completed
                status = job_info.get("status", {})
                if status:
                    completed = status.get("completed", False)
                    status_str = status.get("status_str", "")
                    
                    if completed:
                        if status_str == "success":
                            # Job completed successfully but we didn't find tags yet
                            if outputs:
                                print("Job completed successfully but no tags found in outputs")
                                print(f"Available outputs: {json.dumps(outputs, indent=2)}")
                                # Return whatever outputs we have
                                return {"outputs": outputs}
                            else:
                                print("Job completed but no outputs available yet")
                        else:
                            print(f"Job completed with status: {status_str}")
                            return None
            else:
                print(f"Job {job_id} found in history but unexpected response format, data: {job_data}")
        
        except Exception as e:
            print(f"Error checking job status: {str(e)}")
        
        # Wait before next poll
        time.sleep(poll_interval)
    
    print(f"Timeout reached after {timeout} seconds")
    return None


def tag_image(image_url: str, max_attempts: int = 3, **kwargs) -> Optional[Dict[str, Any]]:
    """
    Tag an image with retry logic
    
    Args:
        image_url: URL of the image to tag
        max_attempts: Maximum number of tagging attempts
        **kwargs: Additional arguments for get_image_tags
        
    Returns:
        Dictionary of image tags, or None if tagging failed
    """
    for attempt in range(max_attempts):
        try:
            print(f"Tagging attempt {attempt+1}/{max_attempts}")
            print(f"Image URL: {image_url}")
            
            # Call the ComfyUI tagging function
            result = get_image_tags(image_url, **kwargs)
            
            if result:
                return result
            print(f"Attempt {attempt+1} failed to return valid tags")
            
        except Exception as e:
            print(f"Error during tagging attempt {attempt+1}: {str(e)}")
            import traceback
            traceback.print_exc()
        
        # Only sleep if we're going to retry
        if attempt < max_attempts - 1:
            backoff = 2 ** attempt
            print(f"Retrying in {backoff} seconds...")
            time.sleep(backoff)
    
    print(f"Failed to tag image after {max_attempts} attempts")
    return None


# Example usage:
if __name__ == "__main__":
    # Example image URL
    test_image_url = "https://scontent-dfw5-1.cdninstagram.com/v/t39.30808-6/479057926_613417618287297_1751938931122809423_n.jpg?stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE3MTF4MjA0OC5zZHIuZjMwODA4LmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QF9dRTzTGD_lefaDT1Mqz4lR6Bu7JW-ThSDeXBH81KJ2HA-jn3uF9vRGtEmY6LFDMQ&_nc_ohc=xS_ajUFYFFEQ7kNvgHi9h5Y&_nc_gid=d81vSigMucAMuP8ftgq4Ug&edm=APoiHPcAAAAA&ccb=7-5&ig_cache_key=MzI1ODYxOTkwMDcwNjYwNDM3MA%3D%3D.3-ccb7-5&oh=00_AYFJQ2QxAh3r7tZqGbihRFVRyOQKJ5ySVlh198uIu1aJrA&oe=67F0B954&_nc_sid=22de04"
    
    # Tag the image
    tags_result = tag_image(
        image_url=test_image_url,
        workflow_file="/Users/l_y_o/Work/happyface/pseo/image-tagger-api.json"
    )
    
    if tags_result:
        print(f"Image tags: {json.dumps(tags_result, indent=2)}")
    else:
        print("Image tagging failed")

Tagging attempt 1/3
Image URL: https://scontent-dfw5-1.cdninstagram.com/v/t39.30808-6/479057926_613417618287297_1751938931122809423_n.jpg?stp=dst-jpg_e35_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkZFRUQuaW1hZ2VfdXJsZ2VuLjE3MTF4MjA0OC5zZHIuZjMwODA4LmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=109&_nc_oc=Q6cZ2QF9dRTzTGD_lefaDT1Mqz4lR6Bu7JW-ThSDeXBH81KJ2HA-jn3uF9vRGtEmY6LFDMQ&_nc_ohc=xS_ajUFYFFEQ7kNvgHi9h5Y&_nc_gid=d81vSigMucAMuP8ftgq4Ug&edm=APoiHPcAAAAA&ccb=7-5&ig_cache_key=MzI1ODYxOTkwMDcwNjYwNDM3MA%3D%3D.3-ccb7-5&oh=00_AYFJQ2QxAh3r7tZqGbihRFVRyOQKJ5ySVlh198uIu1aJrA&oe=67F0B954&_nc_sid=22de04
Submitting tagging job attempt 1/3...
Successfully submitted job with ID: 337e6c19-bb63-499e-9112-4056a5892ffb
Waiting for job 337e6c19-bb63-499e-9112-4056a5892ffb to complete...
Job 337e6c19-bb63-499e-9112-4056a5892ffb not found in history yet. Waiting...
Job 337e6c19-bb63-499e-9112-4056a5892ffb not found in history yet. Waiting...
Successfully extracted image tags
Image tags: {
  "tag

In [ ]:
# iter over the images and tag them
from tqdm import tqdm
image_info = {}
for image in tqdm(images):
    tags = tag_image(image)
    image_info[image] = tags


In [156]:

# save the images with tags
json.dump(images, open('/Users/l_y_o/Work/happyface/pseo/ai_girls_sample_images_tagged.json', 'w'))

In [216]:
ai_girls_types = [
  'nude girl',
  'sexy girl',
  'hot girl',
  'little girl',
  'teen girl',
  'nn girl',
  'bikini girl',
  'anime girl',
  'tween girl', 
  'girls porn',
  'young girl',
  'girl in panties',
  'cute girl',
  'girl in underwear',
  'girls kissing',
  'school girls',
  'asign girls',
  'girl in lingerie',
  'girl satin',
  'muscle girl',
  'beach girl',
  'topless girl',
  'petite girl',
  'pregnant girl',
  'valhalla girl', 
  'asian girls',
]

In [158]:
import json
import time
import requests
import random
import os
import re
from typing import Optional, Dict, Any


def generate_image_with_comfyui(
    prompt: str,
    comfy_server_url: str = "http://72.70.187.181:50346",
    workflow_file: str = "/Users/l_y_o/Work/happyface/pseo/flux-nsfw-upload-api.json",
    seed: Optional[int] = None,
    max_retries: int = 3,
    poll_interval: float = 2.0,
    timeout: int = 300
) -> Optional[str]:
    """
    Generate an image using ComfyUI workflow with the given prompt.
    The workflow will upload the image to Supabase storage.
    
    Args:
        prompt: The text prompt to use for image generation
        comfy_server_url: URL of the ComfyUI server
        workflow_file: Path to the workflow JSON file
        seed: Random seed for image generation (if None, a random seed will be used)
        max_retries: Maximum number of retries for the job submission
        poll_interval: Time in seconds between status check polls
        timeout: Maximum time in seconds to wait for job completion
        
    Returns:
        String URL of the generated image in Supabase, or None if generation failed
    """
    # Load the workflow
    try:
        with open(workflow_file, 'r') as f:
            workflow = json.load(f)
    except Exception as e:
        print(f"Error loading workflow file: {str(e)}")
        return None
    
    # Modify the prompt in the workflow
    # Find the CLIPTextEncode node (node 6 in the workflow)
    for node_id, node in workflow.items():
        if node.get("class_type") == "CLIPTextEncode":
            # Replace the prompt
            node["inputs"]["text"] = prompt
            break
    
    # Generate a random seed if not provided
    if seed is None:
        seed = random.randint(0, 2**32 - 1)
    
    # Find the RandomNoise node and update the seed
    for node_id, node in workflow.items():
        if node.get("class_type") == "RandomNoise":
            node["inputs"]["noise_seed"] = seed
            break
    
    # Prepare the API request
    prompt_api_url = f"{comfy_server_url}/prompt"
    
    # Prepare the payload
    payload = {
        "prompt": workflow,
        "client_id": f"comfy_generator_{int(time.time())}"
    }
    
    # Submit the job
    job_id = None
    for attempt in range(max_retries):
        try:
            print(f"Submitting generation job attempt {attempt+1}/{max_retries}...")
            response = requests.post(prompt_api_url, json=payload)
            response.raise_for_status()
            response_data = response.json()
            job_id = response_data.get("prompt_id")
            if job_id:
                print(f"Successfully submitted job with ID: {job_id}")
                break
        except Exception as e:
            print(f"Attempt {attempt+1}/{max_retries} - Error submitting job: {str(e)}")
            if attempt < max_retries - 1:
                sleep_time = 2 ** attempt  # Exponential backoff
                print(f"Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
    
    if not job_id:
        print("Failed to submit job after all retries")
        return None
    
    # Poll for job completion using the job history endpoint
    start_time = time.time()
    job_api_url = f"{comfy_server_url}/history/{job_id}"
    queue_api_url = f"{comfy_server_url}/queue"
    
    print(f"Waiting for job {job_id} to complete...")
    
    while time.time() - start_time < timeout:
        try:
            # Check queue status to see if our job is running
            queue_response = requests.get(queue_api_url)
            if queue_response.status_code == 200:
                queue_data = queue_response.json()
                running = queue_data.get("running", [])
                queue_remaining = queue_data.get("queue_remaining", 0)
                
                if running:
                    for job in running:
                        # Check if our job is running
                        if job.get("prompt_id") == job_id:
                            progress = job.get("progress", 0)
                            print(f"Job running: {progress*100:.1f}% complete")
                elif queue_remaining > 0:
                    print(f"Job queued. Position in queue: {queue_remaining}")
            
            # Get job status from history
            job_response = requests.get(job_api_url)
            
            # Skip if job not found yet
            if job_response.status_code == 404:
                print(f"Job {job_id} not found in history yet. Waiting...")
                time.sleep(poll_interval)
                continue
                
            job_response.raise_for_status()
            job_data = job_response.json()

            if job_data == {}:
                print(f"Job {job_id} not found in history yet. Waiting...")
                time.sleep(poll_interval)
                continue
            
            # The response has the job_id as the key
            if job_id in job_data:
                job_info = job_data[job_id]
                
                # Check for outputs
                outputs = job_info.get("outputs", {})
                if outputs:
                    # Check node 57 (Display Any rgthree)
                    if "57" in outputs and outputs["57"]:
                        # For text output that contains the URL
                        if "text" in outputs["57"]:
                            for text_item in outputs["57"]["text"]:
                                if isinstance(text_item, str) and "http" in text_item:
                                    # Extract URL
                                    url_pattern = r'https?://\S+'
                                    match = re.search(url_pattern, text_item)
                                    if match:
                                        supabase_url = match.group(0)
                                        print(f"Successfully generated image: {supabase_url}")
                                        return supabase_url
                    
                    # Check for Supabase uploader output directly (node 55)
                    if "55" in outputs:
                        print("Found Supabase uploader output")
                        # Extract URL from Supabase uploader output
                        if "url" in outputs["55"]:
                            supabase_url = outputs["55"]["url"]
                            print(f"Successfully generated image: {supabase_url}")
                            return supabase_url
                        elif isinstance(outputs["55"], str) and "http" in outputs["55"]:
                            # Try to extract URL from string
                            url_pattern = r'https?://\S+'
                            match = re.search(url_pattern, outputs["55"])
                            if match:
                                supabase_url = match.group(0)
                                print(f"Successfully generated image: {supabase_url}")
                                return supabase_url
                        else:
                            # If the URL isn't in an expected format, return whatever is available
                            print(f"Unexpected format from Supabase uploader: {outputs['55']}")
                            return str(outputs["55"])
                
                # Check if job is completed
                status = job_info.get("status", {})
                if status:
                    completed = status.get("completed", False)
                    status_str = status.get("status_str", "")
                    
                    if completed:
                        if status_str == "success":
                            # Job completed successfully but we didn't find URL yet
                            if outputs:
                                print("Job completed successfully but no URL found in outputs")
                                print(f"Available outputs: {json.dumps(outputs, indent=2)}")
                            else:
                                print("Job completed but no outputs available yet")
                        else:
                            print(f"Job completed with status: {status_str}")
                            return None
            else:
                print(f"Job {job_id} found in history but unexpected response format, data: {job_data}")
        
        except Exception as e:
            print(f"Error checking job status: {str(e)}")
        
        # Wait before next poll
        time.sleep(poll_interval)
    
    print(f"Timeout reached after {timeout} seconds")
    return None


def generate_image(prompt: str, max_attempts: int = 3, **kwargs) -> Optional[str]:
    """
    Generate an image with retry logic
    
    Args:
        prompt: The text prompt for image generation
        max_attempts: Maximum number of generation attempts
        **kwargs: Additional arguments for generate_image_with_comfyui
        
    Returns:
        String URL of the generated image, or None if generation failed
    """
    for attempt in range(max_attempts):
        try:
            print(f"Generation attempt {attempt+1}/{max_attempts}")
            print(f"Prompt: {prompt}")
            
            # Call the ComfyUI generation function
            result = generate_image_with_comfyui(prompt, **kwargs)
            
            if result:
                return result
            print(f"Attempt {attempt+1} failed to return a valid URL")
            
        except Exception as e:
            print(f"Error during generation attempt {attempt+1}: {str(e)}")
            import traceback
            traceback.print_exc()
        
        # Only sleep if we're going to retry
        if attempt < max_attempts - 1:
            backoff = 2 ** attempt
            print(f"Retrying in {backoff} seconds...")
            time.sleep(backoff)
    
    print(f"Failed to generate image after {max_attempts} attempts")
    return None


# Example usage:
if __name__ == "__main__":
    # Example prompt
    test_prompt = "A beautiful woman at the beach, wearing a swimsuit"
    
    # Generate the image
    image_url = generate_image(
        prompt=test_prompt,
        comfy_server_url="http://72.70.187.181:50346/",
        workflow_file="/Users/l_y_o/Work/happyface/pseo/flux-nsfw-upload-api.json"
    )
    
    if image_url:
        print(f"Final image URL: {image_url}")
    else:
        print("Image generation failed")

Generation attempt 1/3
Prompt: A beautiful woman at the beach, wearing a swimsuit
Submitting generation job attempt 1/3...
Successfully submitted job with ID: 680200b4-76a8-43ea-8459-e863f94928fe
Waiting for job 680200b4-76a8-43ea-8459-e863f94928fe to complete...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f94928fe not found in history yet. Waiting...
Job 680200b4-76a8-43ea-8459-e863f9

In [206]:

with open('/Users/l_y_o/Work/happyface/pseo/ai_girls_types_images.json', 'r') as f:
    types_images = json.load(f)
    print(f"Loaded {len(types_images)} types")
for ai_girl_type in tqdm(ai_girls_types):
    print(f"type {ai_girl_type} not done: {ai_girl_type not in types_images or len(types_images[ai_girl_type]) < 10:}")






Loaded 17 types


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 55072.27it/s]

type nude girl not done: False
type sexy girl not done: False
type hot girl not done: False
type little girl not done: False
type teen girl not done: False
type nn girl not done: False
type bikini girl not done: False
type anime girl not done: False
type tween girl not done: False
type girls porn not done: False
type young girl not done: False
type girl in panties not done: False
type cute girl not done: False
type girl in underwear not done: False
type girls kissing not done: False
type school girls not done: False
type asign girls not done: True
type girl in lingerie not done: True
type girl satin not done: True
type muscle girl not done: True
type beach girl not done: True
type topless girl not done: True
type petite girl not done: True
type pregnant girl not done: True
type valhalla girl not done: True


In [217]:
# iter over the types and generate images, gen 6 images for each type
# types_images = {}
for ai_girl_type in tqdm(ai_girls_types):
    if ai_girl_type not in types_images:
        types_images[ai_girl_type] = []

    if ai_girl_type in types_images and len(types_images[ai_girl_type]) >= 10:
        print(f"type {ai_girl_type} already has 10 images")
        continue
    while len(types_images[ai_girl_type]) < 10:
        try:
            image_url = generate_image(
                prompt=ai_girl_type + ", beautiful girl",
                comfy_server_url="http://193.143.121.70:39267/",
                    workflow_file="/Users/l_y_o/Work/happyface/pseo/flux-nsfw-upload-api.json"
                )
            types_images[ai_girl_type].append(image_url)
        except Exception as e:
            print(f"Error generating image for {ai_girl_type}: {str(e)}")
            continue

    # save the types_images
    with open('/Users/l_y_o/Work/happyface/pseo/ai_girls_types_images.json', 'w') as f:
        json.dump(types_images, f)
        print(f"Saved {len(types_images[ai_girl_type])} images for {ai_girl_type}")



  0%|                                                                                                                                                                                                                                                                          | 0/26 [00:00<?, ?it/s]

type nude girl already has 10 images
type sexy girl already has 10 images
type hot girl already has 10 images
type little girl already has 10 images
type teen girl already has 10 images
type nn girl already has 10 images
type bikini girl already has 10 images
type anime girl already has 10 images
type tween girl already has 10 images
type girls porn already has 10 images
type young girl already has 10 images
type girl in panties already has 10 images
type cute girl already has 10 images
type girl in underwear already has 10 images
type girls kissing already has 10 images
type school girls already has 10 images
type asign girls already has 10 images
type girl in lingerie already has 10 images
type girl satin already has 10 images
type muscle girl already has 10 images
type beach girl already has 10 images
type topless girl already has 10 images
type petite girl already has 10 images
type pregnant girl already has 10 images
type valhalla girl already has 10 images
Generation attempt 1/3


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [10:56<00:00, 25.24s/it]

Successfully generated image: https://bjqbwtqznzladztznntj.supabase.co/storage/v1/object/public/images/ai-girls-gend/comfy_90b5dedc-dda5-4bf1-9073-14aba1e6d0d3.png?
Saved 10 images for asian girls


In [218]:

# save the types_images
with open('/Users/l_y_o/Work/happyface/pseo/ai_girls_types_images.json', 'w') as f:
    json.dump(types_images, f)
    print(f"Saved {len(types_images[ai_girl_type])} images for {ai_girl_type}")


Saved 10 images for asian girls


In [164]:
len(types_images), len(ai_girls_types)

(17, 22)

# add missing faq dict 

# gen landing contents

In [219]:
examples = []
with open("/Users/l_y_o/Work/happyface/app/tools/[generator-title]/enhanced_results_with_prompts_nsfw_image_gend.json", "r") as f:
    examples = json.load(f)

landing_keys = ["title", "subtitle", "how_to_use_title", "how_to_step1_title", "how_to_step1_content", "how_to_step2_title", "how_to_step2_content", "how_to_step3_title", "how_to_step3_content", 
                "feature1_title", "feature1_content", "feature2_title", "feature2_content", "feature3_title", "feature3_content", "cta_title", "cta_content", "faq_dict", "title_seo", "description_seo"]

example_prompts = []
for i in range(2,4):
    prompt = ""
    for key in landing_keys:
        if key == "faq_dict" and key not in examples[i]:
            print(f"no faq_dict for {examples[i]['title']}")
            continue
        prompt += "{key}: {value}, ".format(key=key, value=examples[i][key])

    example_prompts.append(prompt)


example_prompts







['title: AI Milf Generator, subtitle: Easily create your ideal AI generated milf using this AI tool. With this simple MILF generator, you can produce realistic photos in just a few seconds. No technical skills needed—just enter your preferences and let the AI do the work., how_to_use_title: How to use AI Milf Generator:, how_to_step1_title: Describe Your Image, how_to_step1_content: Start by entering a detailed description of the image you want to create in the prompt box. Be clear and specific, including important keywords to help the AI generate the image you envision. For example, you can describe a landscape, a portrait, or an abstract concept. Once you\'re satisfied with your description, click the "Generate" button., how_to_step2_title: Generation Process, how_to_step2_content: After you click "Generate," the AI Image Generator will process your prompt and begin creating your AI generated milf. This may take a few moments, depending on the complexity of your request and the curre

In [ ]:
# use deep seek to gen landing contents

KeyError: 'faq_dict'

In [223]:
import json
import time
import os
from openai import OpenAI
from typing import Dict, List, Any, Optional
from tqdm import tqdm
import pandas as pd

# Define the types of AI Girls from the notebook
ai_girls_types = [
  'nude girl',
  'sexy girl',
  'hot girl',
  'little girl',
  'teen girl',
  'nn girl',
  'bikini girl',
  'anime girl',
  'tween girl', 
  'girls porn',
  'young girl',
  'girl in panties',
  'cute girl',
  'girl in underwear',
  'girls kissing',
  'school girls',
  'asign girls',
  'girl in lingerie',
  'girl satin',
  'muscle girl',
  'beach girl',
  'topless girl',
  'petite girl',
  'pregnant girl',
  'valhalla girl', 
  'asian girls',
]

# Define landing page keys
landing_keys = [
    "title", "subtitle", "how_to_use_title", "how_to_step1_title", "how_to_step1_content",
    "how_to_step2_title", "how_to_step2_content", "how_to_step3_title", "how_to_step3_content",
    "feature1_title", "feature1_content", "feature2_title", "feature2_content",
    "feature3_title", "feature3_content", "cta_title", "cta_content", "faq_dict",
    "title_seo", "description_seo"
]

# Example landing page content for few-shot learning
example_landing_pages = [
    {
        "title": "AI Milf Generator",
        "subtitle": "Easily create your ideal AI generated milf using this AI tool. With this simple MILF generator, you can produce realistic photos in just a few seconds. No technical skills needed—just enter your preferences and let the AI do the work.",
        "how_to_use_title": "How to use AI Milf Generator:",
        "how_to_step1_title": "Describe Your Image",
        "how_to_step1_content": "Start by entering a detailed description of the image you want to create in the prompt box. Be clear and specific, including important keywords to help the AI generate the image you envision. For example, you can describe a landscape, a portrait, or an abstract concept. Once you're satisfied with your description, click the \"Generate\" button.",
        "how_to_step2_title": "Generation Process",
        "how_to_step2_content": "After you click \"Generate,\" the AI Image Generator will process your prompt and begin creating your AI generated milf. This may take a few moments, depending on the complexity of your request and the current system load.",
        "how_to_step3_title": "Download Your Image",
        "how_to_step3_content": "When the AI finishes generating your image, it will appear on the screen. You can then download it to your device for personal or commercial use. Share your unique AI-generated creation with others or incorporate it into your projects.",
        "feature1_title": "Create Your Perfect MILF AI Art",
        "feature1_content": "No need to search endlessly for the right image anymore. With our AI MILF generator, you can create custom MILF art that matches your specific preferences. Simply provide a detailed description, and the AI will generate a high-quality image suited to your needs.",
        "feature2_title": "Simple and User-Friendly",
        "feature2_content": "Our AI MILF generator is easy to use for everyone. There's no need for technical expertise—just input your description and let the AI do the work. Quickly produce stunning MILF AI images without any fuss.",
        "feature3_title": "High-Quality Results Every Time",
        "feature3_content": "Experience the power of AI in creating detailed and realistic MILF images. Our advanced AI technology guarantees high-quality results that look professional and lifelike. Whether for personal use or creative projects, create the best quality image every time.",
        "cta_title": "More Than Just a AI Milf Generator",
        "cta_content": "BasedLabs isn't just an AI Milf Generator. Sure, you can download AI-generated images in seconds, but there's a lot more. You can use our built-in video editing app to turn your photos into videos, and even edit audio. It's a full-featured professional video editing software that lets you create amazing videos in minutes, without needing any prior experience. Give BasedLabs a try today and see how AI can enhance your images, videos, and audio!",
        "faq_dict": {
            "How can I get started with the AI MILF generator?": "To get started, visit the generator's platform, sign up or log in, and follow the instructions to create your desired images.",
            "Does the AI MILF generator cost anything?": "Pricing details vary depending on the features and usage. Check the platform's pricing page for more information.",
            "How quickly are images generated?": "Images are generated almost instantly, typically within a few seconds, depending on the complexity of your request.",
            "What file types are available for export?": "The generator supports multiple file formats including JPEG, PNG, and others commonly used for image sharing and editing.",
            "How is my data kept secure while using the generator?": "Your data is kept secure through encryption and adherence to industry-best practices for privacy protection."
        },
        "title_seo": "AI Milf Generator: Create Stunning AI Images",
        "description_seo": "Discover the AI Milf Generator to craft stunning, customized AI imagery. Easy-to-use and perfect for creative projects. Try it now!"
    },
    {
        "title": "AI Waifu Generator",
        "subtitle": "Explore the capabilities of the AI waifu generator, enabling you to bring your ideal anime waifu to life in any pose you can imagine.",
        "how_to_use_title": "How to use AI Waifu Generator:",
        "how_to_step1_title": "Define Your Waifu",
        "how_to_step1_content": "Input your idea into the prompt box, using specific keywords to define the style and characteristics of your anime waifu. Once you're ready, click the \"Generate Images\" button to start the process.",
        "how_to_step2_title": "Wait for Processing",
        "how_to_step2_content": "Allow the AI a moment to interpret your description and generate the image. Please be patient during this brief period.",
        "how_to_step3_title": "Review the Artwork",
        "how_to_step3_content": "After the AI completes its task, you can view the detailed artwork based on your concept. This image is yours to use freely, including for commercial purposes.",
        "feature1_title": "Effortless Waifu Creation",
        "feature1_content": "Bring your dream anime waifu to life with our waifu AI generator by simply describing her in words. No art skills required—let our waifu generator AI translate your ideas into stunning artwork that matches your vision perfectly.",
        "feature2_title": "Customizable Poses",
        "feature2_content": "Design your anime waifu in any pose you desire using our anime waifu AI generator. Whether you're aiming for something playful, serious, or dynamic, our AI tool can craft the perfect image to suit your needs.",
        "feature3_title": "Instant Results",
        "feature3_content": "Experience the satisfaction of seeing your concept materialize within seconds with our waifu AI art generator. Our powerful waifu generator AI processes your descriptions quickly, providing you with detailed, high-quality images ready for use, including NSFW AI waifu generator options if desired.",
        "cta_title": "More Than Just an AI Waifu Generator",
        "cta_content": "BasedLabs offers a diverse set of AI-powered tools that cater to all your creative needs, beyond just waifu generation. Utilize our advanced AI models for image creation, photo editing, and video transformation to enhance your visual content. Our user-friendly tools are built for efficiency and innovation, supporting professionals, content creators, and anime enthusiasts in reaching new creative heights.",
        "faq_dict": {
            "What is an AI Waifu Generator?": "An AI Waifu Generator is an advanced AI tool that creates customized anime-style character images based on user descriptions, allowing fans to bring their ideal waifu concepts to life with remarkable detail and accuracy.",
            "Can I customize my waifu's appearance?": "Yes, you can fully customize your waifu's appearance by providing detailed descriptions of characteristics like hair color, eye shape, outfit style, pose, and background elements in your generation prompt.",
            "How realistic do the generated waifus look?": "The generated waifus can look remarkably realistic within anime aesthetics, with high-quality rendering of features, textures, and lighting that create visually appealing and cohesive character designs.",
            "Can I use these images commercially?": "Most AI Waifu Generators allow for commercial use of generated images, though specific terms may vary by platform. Always check the terms of service for the particular tool you're using.",
            "Is the waifu generator suitable for beginners?": "Absolutely. The waifu generator features an intuitive interface designed for users of all experience levels. No technical or artistic expertise is required to create stunning anime character images."
        },
        "title_seo": "AI Waifu Generator - Create Your Perfect Anime Companion",
        "description_seo": "Design custom AI waifus with advanced tools. Generate your ideal anime companion with unique features and styles. Try it now!"
    
    }
]

def setup_perplexity_client(api_key: str) -> OpenAI:
    """
    Set up the OpenAI client to use with the Perplexity API.
    
    Args:
        api_key: Perplexity API key
        
    Returns:
        OpenAI client configured for Perplexity
    """
    return OpenAI(
        api_key=api_key,
        base_url="https://api.perplexity.ai"
    )

def create_landing_page_prompt(girl_type: str, examples: List[Dict[str, Any]]) -> str:
    """
    Create a prompt for generating landing page content for a specific type of AI girl.
    
    Args:
        girl_type: The type of AI girl (e.g., "nude girl", "anime girl")
        examples: List of example landing pages for few-shot learning
        
    Returns:
        A string containing the prompt for the model
    """
    # Convert examples to a readable format for few-shot learning
    example_text = json.dumps(examples, indent=2)
    
    prompt = f"""Generate a complete landing page content for an "AI {girl_type.title()} Generator" website. 
    
The landing page should include professional marketing content that focuses on the AI image generation capabilities.
The tone should be professional and appeal to users interested in AI image generation technology.

The landing page should include the following sections:
1. Title and subtitle
2. How to use the generator (with 3 steps)
3. Feature highlights (3 features)
4. Call to action
5. FAQ (5 questions and answers)
6. SEO optimized title and description

Here are examples of landing pages for similar AI generators to use as reference:
{example_text}

Return the content as a JSON object with the following keys:
{json.dumps(landing_keys, indent=2)}

For the faq_dict, provide a dictionary with 5 question-answer pairs.

For title_seo, create an SEO-optimized title tag that includes keywords related to the AI generator type, with a maximum of 60 characters.

For description_seo, create a compelling meta description that summarizes the tool's purpose and benefits, with a maximum of 160 characters.

Make sure all content is professional, tasteful, and focuses on the AI technology's capabilities without being explicit, even if the theme might suggest adult content.
"""
    return prompt

def generate_landing_page(client: OpenAI, girl_type: str, examples: List[Dict[str, Any]], 
                         model: str = "r1-1776", max_retries: int = 3) -> Optional[Dict[str, Any]]:
    """
    Generate landing page content using the Perplexity API.
    
    Args:
        client: OpenAI client configured for Perplexity
        girl_type: The type of AI girl
        examples: Example landing pages for few-shot learning
        model: Model to use for generation
        max_retries: Number of retries in case of errors
        
    Returns:
        Dictionary containing the landing page content, or None if generation failed
    """
    prompt = create_landing_page_prompt(girl_type, examples)
    
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a professional copywriter specializing in creating marketing content for AI image generation websites."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=3000,
            )
            
            content = response.choices[0].message.content
            
            # Parse the JSON response
            try:
                # Extract JSON content if not already properly formatted
                if not content.strip().startswith('{'):
                    # Try to find JSON in the response
                    json_start = content.find('{')
                    json_end = content.rfind('}') + 1
                    if json_start >= 0 and json_end > json_start:
                        content = content[json_start:json_end]
                
                result = json.loads(content)
                return result
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON for {girl_type}: {e}")
                print(f"Raw content: {content}")
                
                if attempt < max_retries - 1:
                    print(f"Retrying ({attempt+1}/{max_retries})...")
                    time.sleep(2)
                else:
                    return {"error": "Failed to parse JSON", "raw_content": content}
                
        except Exception as e:
            print(f"Error generating content for {girl_type}: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying ({attempt+1}/{max_retries})...")
                time.sleep(2)
    
    return None

def main(results):
    """
    Main function to generate landing pages for all AI girl types.
    """
    # Get API key from environment or prompt user
    api_key = os.environ.get("PERPLEXITY_API_KEY")
    if not api_key:
        api_key = input("Enter your Perplexity API key: ")
    
    # Set up the client
    client = setup_perplexity_client(api_key)
    
    # Initialize results storage
    # results = {}
    
    # Generate landing pages for each type
    for girl_type in tqdm(ai_girls_types, desc="Generating landing pages"):
        if girl_type in results:
            print(f"skipping {girl_type} because it already exists")
            continue
        print(f"\nGenerating landing page for: {girl_type}")
        landing_page = generate_landing_page(client, girl_type, example_landing_pages)
        
        if landing_page:
            results[girl_type] = landing_page
            print(f"Successfully generated content for {girl_type}")
        else:
            print(f"Failed to generate content for {girl_type}")
    
    # Save the results to a JSON file
    with open("ai_girls_landing_pages.json", "w") as f:
        json.dump(results, f, indent=2)
    
    print(f"\nGenerated {len(results)} landing pages out of {len(ai_girls_types)} types")
    print("Results saved to ai_girls_landing_pages.json")
    
    # Also convert to a DataFrame for easier viewing
    df_rows = []
    for girl_type, content in results.items():
        if isinstance(content, dict) and "error" not in content:
            row = {"type": girl_type}
            for key in landing_keys:
                if key == "faq_dict" and isinstance(content.get(key), dict):
                    row[key] = json.dumps(content.get(key))
                else:
                    row[key] = content.get(key)
            df_rows.append(row)
    
    if df_rows:
        df = pd.DataFrame(df_rows)
        df.to_csv("ai_girls_landing_pages.csv", index=False)
        print("CSV file also created: ai_girls_landing_pages.csv")


In [220]:
results

[{'title': 'DnD AI Art Generator',
  'subtitle': 'Try our DnD AI art generator to bring your DnD characters to life. Experience the world of Dungeons & Dragons as your unique creations are visualized in stunning detail.',
  'how_to_use_title': 'How to use DnD AI Art Generator:',
  'how_to_step1_image': 'https://cdn.basedlabs.ai/9f9b8f90-1323-11ef-a1ff-372c484d0bb7.png',
  'how_to_step2_image': 'https://cdn.basedlabs.ai/a5ae6e70-1323-11ef-a1ff-372c484d0bb7.png',
  'how_to_step3_image': 'https://cdn.basedlabs.ai/5e354960-770f-11ef-be95-bdc3ce781d17.png',
  'how_to_step1_title': 'Enter Your Prompt',
  'how_to_step2_title': 'Let the AI Work',
  'how_to_step3_title': 'Explore Your Artwork',
  'how_to_step1_content': 'Begin by entering a description of your character in the prompt area. You can include keywords to specify the style and mood. Then, click the "Create Images" button.',
  'how_to_step2_content': 'Wait a couple seconds for the AI to interpret your text and generate an image. Our 

In [225]:

with open("ai_girls_landing_pages.json", "r") as f:
    ai_girls_results = json.load(f)

ai_girls_results

{'nude girl': {'title': 'AI Nude Girl Generator',
  'subtitle': 'Create custom AI-generated figures with advanced image generation technology. Craft detailed, realistic images in seconds through intuitive text prompts.',
  'how_to_use_title': 'How to Use the AI Nude Girl Generator:',
  'how_to_step1_title': 'Describe Your Vision',
  'how_to_step1_content': 'Enter a detailed text prompt specifying your desired figure’s pose, style, and artistic direction. Use descriptive keywords to guide the AI’s creative process for optimal results.',
  'how_to_step2_title': 'AI Processing',
  'how_to_step2_content': 'Our advanced neural network analyzes your input to generate a high-resolution image. Processing typically takes 10-20 seconds, depending on complexity and server load.',
  'how_to_step3_title': 'Export & Utilize',
  'how_to_step3_content': 'Download your AI-generated image in PNG or JPEG format. Use it for artistic projects, 3D modeling references, or creative concept development with fu

In [226]:
os.environ["PERPLEXITY_API_KEY"] = "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"

if __name__ == "__main__":
    main(ai_girls_results)

Generating landing pages:   0%|                                                                                                                                                                                                                                                | 0/26 [00:00<?, ?it/s]

skipping nude girl because it already exists
skipping sexy girl because it already exists
skipping hot girl because it already exists
skipping little girl because it already exists
skipping teen girl because it already exists
skipping nn girl because it already exists
skipping bikini girl because it already exists
skipping anime girl because it already exists
skipping tween girl because it already exists
skipping girls porn because it already exists
skipping young girl because it already exists
skipping girl in panties because it already exists
skipping cute girl because it already exists
skipping girl in underwear because it already exists
skipping girls kissing because it already exists
skipping school girls because it already exists
skipping asign girls because it already exists
skipping girl in lingerie because it already exists
skipping girl satin because it already exists
skipping muscle girl because it already exists
skipping beach girl because it already exists
skipping topless

2025-04-01 15:57:15,067:INFO - HTTP Request: POST https://api.perplexity.ai/chat/completions "HTTP/1.1 200 OK"
Generating landing pages: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:21<00:00,  1.20it/s]

Successfully generated content for asian girls

Generated 26 landing pages out of 26 types
Results saved to ai_girls_landing_pages.json
CSV file also created: ai_girls_landing_pages.csv
